In [1]:
# Importing libraries
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import StandardScaler as SS
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report as clsr, accuracy_score as accs
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [2]:
# Load the Iris dataset
iris = load_iris()
X = iris.data  # Features: Sepal, petal lengths & widths
y = iris.target  # Labels: Setosa, Versicolour, Virginica

# Quick look at the features and labels
print("Features:\n", X[:5])
print("\nLabels:\n", y[:5])


Features:
 [[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]

Labels:
 [0 0 0 0 0]


In [3]:
# Split dataset into train and test
X_train, X_test, y_train, y_test = tts(X, y, test_size=0.2, random_state=42)
print(f"Training set size: {len(X_train)}\nTesting set size: {len(X_test)}")


Training set size: 120
Testing set size: 30


In [4]:
# Feature scaling
sc = SS()
X_train = sc.fit_transform(X_train)  # Fit to train, transform train
X_test = sc.transform(X_test)        # Only transform test (important!)


In [5]:
# Logistic Regression Model
lr = LogisticRegression(max_iter=250)  # Cranked iterations to 250, just in case
lr.fit(X_train, y_train)               # Training... easy peasy
y_pred_logreg = lr.predict(X_test)     # Predictions

# Evaluate Logistic Regression
print("LogReg Accuracy:", accs(y_test, y_pred_logreg))
print("LogReg Classification Report:\n", clsr(y_test, y_pred_logreg))
print("Confusion Matrix (LogReg):\n", pd.crosstab(y_test, y_pred_logreg))  # Sneaky Pandas!


LogReg Accuracy: 1.0
LogReg Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

Confusion Matrix (LogReg):
 col_0   0  1   2
row_0           
0      10  0   0
1       0  9   0
2       0  0  11


In [6]:
# Neural Network with TensorFlow
nn_model = Sequential()
nn_model.add(Dense(units=8, activation='relu', input_dim=4))  # 4 features in, 8 neurons
nn_model.add(Dense(units=8, activation='relu'))               # Another layer for fun
nn_model.add(Dense(units=3, activation='softmax'))            # Output layer for 3 classes

# Compile the NN model
nn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the NN model
nn_model.fit(X_train, y_train, epochs=30, verbose=1, validation_split=0.2)  # Split 20% for validation


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 257ms/step - accuracy: 0.1484 - loss: 1.4485 - val_accuracy: 0.2083 - val_loss: 1.4295
Epoch 2/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.1055 - loss: 1.4433 - val_accuracy: 0.2083 - val_loss: 1.4071
Epoch 3/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.1328 - loss: 1.4017 - val_accuracy: 0.2083 - val_loss: 1.3859
Epoch 4/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.1055 - loss: 1.3743 - val_accuracy: 0.2083 - val_loss: 1.3652
Epoch 5/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.1523 - loss: 1.3371 - val_accuracy: 0.2083 - val_loss: 1.3455
Epoch 6/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.1250 - loss: 1.3281 - val_accuracy: 0.2083 - val_loss: 1.3264
Epoch 7/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.1367 - loss: 1.3133 - val_accuracy: 0.2083 - val_loss: 1.3083
Epoch 8/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.1406 - loss: 1.2690 - val_accuracy: 0.2083 - val_loss: 1.2912

In [7]:
# Evaluate NN
nn_loss, nn_acc = nn_model.evaluate(X_test, y_test, verbose=0)  # Turn off verbose, cleaner output
print("Neural Network Accuracy:", nn_acc)

# Predict with the NN model
nn_preds = nn_model.predict(X_test)
nn_preds_classes = np.argmax(nn_preds, axis=1)  # Converting probabilities to class labels

# Evaluate NN predictions
print("Neural Network Classification Report:\n", clsr(y_test, nn_preds_classes))
print("Confusion Matrix (NN):\n", pd.crosstab(y_test, nn_preds_classes))  # More Pandas tricks


Neural Network Accuracy: 0.36666667461395264
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
Neural Network Classification Report:
               precision    recall  f1-score   support

           0       0.29      0.60      0.39        10
           1       0.62      0.56      0.59         9
           2       0.00      0.00      0.00        11

    accuracy                           0.37        30
   macro avg       0.30      0.39      0.33        30
weighted avg       0.28      0.37      0.31        30

Confusion Matrix (NN):
 col_0   0  1  2
row_0          
0       6  3  1
1       4  5  0
2      11  0  0
